In [2]:
import jieba
import re
from tqdm import tqdm
from sklearn.feature_extraction.text import TfidfVectorizer
import sys
import io
import json
import pickle
from utils import cut_sent


def get_stop_words():
    # 加载停用词表
    with open('data/hit_stopwords.txt', 'r', encoding='utf-8') as f:
        stop_words = [line.strip() for line in f]
    return stop_words

stop_words = get_stop_words()
# 读取文本
corpus = []
with open('data/train_2.json', 'r', encoding='utf8') as f:
    train_list = json.load(f)
    for item in tqdm(train_list):
        fact1 = item['fact1']
        fact2 = item['fact2']
        corpus += [fact1, fact2]


def fit(corpus):
    formatted_corpus = []
    # jieba分词
    for document in corpus:
        words = jieba.cut(document, cut_all=False, HMM=False)
        words = [word for word in words if word not in stop_words and re.match(r'^[\s\u4e00-\u9fa5]*$', word) and word!=' ']
        str1 = ' '.join(words)
        formatted_corpus.append(str1)
    vectorizer = TfidfVectorizer()
    X = vectorizer.fit_transform(formatted_corpus)
    print(X.shape)
    weights = X.toarray()
    with open("tf_idf_model.pkl", 'wb') as fw:
        pickle.dump(vectorizer, fw)
    return X, vectorizer

100%|██████████| 46312/46312 [00:00<00:00, 1106887.65it/s]


In [3]:
X, vectorizer = fit(corpus)

Building prefix dict from the default dictionary ...
Loading model from cache /tmp/jieba.cache
Loading model cost 0.720 seconds.
Prefix dict has been built successfully.


(92624, 32225)


In [22]:
cnt = 0
for it in X.toarray()[1]:
    if it != 0:
        cnt += 1
cnt

121

In [30]:
feature_names = vectorizer.get_feature_names_out()

In [29]:
weights = X.toarray()

In [31]:
def get_keywords(text, feature_names, weights):
    # 使用 jieba 对文本进行分词

    words = jieba.cut(text, cut_all=False, HMM=False)
    # 过滤掉标点符号和停用词
    words = [word for word in words if word not in stop_words and re.match(r'^[\s\u4e00-\u9fa5]*$', word) and word!=' ']
    # print(words)
    # 得到每个词的权重
    # 将每个词和对应的权重放入字典中
    keywords = {feature_names[i]: weights[0][i] for i in range(len(weights[0]))}
    # 按照权重从大到小排序
    keywords = sorted(keywords.items(), key=lambda x: x[1], reverse=True)
    return keywords

In [33]:
keywords = get_keywords("提请假释建议书2016年娄底监狱假字第46号罪犯王康，男，1987年10月23日出生，汉族，初中文化，湖南省湘乡市人，现在娄底监狱服刑。湖南省长沙市开福区人民法院于二0一一年一月十二日作出（2010）开刑初字第475号刑事判决，以该犯犯盗窃罪，判处有期徒刑十一年，并处罚金三万元，剥夺政治权利一年，刑期从2010年6月10日起至2021年6月9日止。在法定期限内没有上诉、抗诉。2011年1月27日交付执行。服刑期间刑期变动情况：2014年7月1日减去有期徒刑一年八个月，剥夺政治权利一年不变，减刑后刑期至2019年10月9日止。该犯在服刑改造期间，确有悔改表现，并已服够法定年限，具体事实如下：该犯自上次减刑以来，能认罪服法，服从管教，能较好地遵守监规监纪。积极参加\"三课\"学习，态度端正，目的明确。在思想教育中，能联系自己的实际，深挖犯罪根源；在文化和技术学习中，认真好学，成绩较好。在劳动改造中，积极肯干。服从管理民警安排，能按时、保质、保量完成自己的任务。经考核，该犯现累计奖励分158.5分。期内获得单项奖励分30.6分，2014年3月27日因被评为2013年度优秀互监组长奖奖励分3分，2015年2月28日因2014年度实现“七个不发生”奖奖励分3.3分，2016年2月18日因2015年度实现“四无”奖奖励分3分，2016年3月21日因2015年度实现“七个不发生”奖奖励分10.3分。期内扣奖励分1分。上述事实，有罪犯考核奖惩统计台账、罪犯奖惩审批表、罪犯减刑评议书等材料证实。综上所述，该犯在服刑期间确有悔改表现，为此", feature_names, weights)
keywords

[('周正', 0.42116190402847997),
 ('岳阳市', 0.4042248667438648),
 ('湖南省', 0.3777514595983071),
 ('一年', 0.24552107620992375),
 ('娄底市', 0.22543191619333278),
 ('教学', 0.17129774025096162),
 ('不变', 0.16997361750098056),
 ('政治权利', 0.14368528236189257),
 ('剥夺', 0.14364150184433647),
 ('裁定', 0.11974692930533376),
 ('一个月', 0.10933382908808441),
 ('减刑', 0.10529942652659598),
 ('作出', 0.10421667117095082),
 ('考核', 0.10192817097365633),
 ('完成', 0.10191469207798687),
 ('娄星区', 0.0978841260081294),
 ('罪犯', 0.09717550510551028),
 ('遵行', 0.0917402164854426),
 ('执字', 0.09146435917644176),
 ('每日', 0.08734733755824838),
 ('专业', 0.08693199903732793),
 ('中级', 0.08652123877831766),
 ('人民法院', 0.08452810097259074),
 ('核对', 0.08316755759643485),
 ('教育', 0.08313445608756191),
 ('刑事', 0.08160389333976499),
 ('奖惩', 0.07703612320093206),
 ('入学', 0.0769831515028594),
 ('审批表', 0.07646614545871176),
 ('管教', 0.07559773586329245),
 ('圆满完成', 0.07554922125130392),
 ('各项', 0.07283411763880804),
 ('三万元', 0.07072547733585183),
 (

In [36]:
def get_summary(text, num_sentences):

    # 对文本进行句子切分
    sentences = re.split(r'[。！？，]', text)
    # 获取文本中的关键词
    keywords = get_keywords(text, feature_names, weights)
    # 对每个句子计算其中包含的重要词的权重总和
    sentence_scores = []
    for sentence in sentences:
        score = 0
        for keyword, weight in keywords:
            if keyword in sentence:
                score += weight
        sentence_scores.append(score)

  # 将权重最大的几个句子连接起来，得到文本的摘要
#   首先调用函数sentence_scores.index(max(sentence_scores))获取权重最大的句子的索引
#   然后使用这个索引从句子列表sentences中取出这个句子并将它加入到结果列表summary中
#   最后将权重最大的句子的权重设为-1,以避免在之后的迭代中再次选中。
    summary = []
    for i in range(num_sentences):
        summary.append(sentences[sentence_scores.index(max(sentence_scores))])
        sentence_scores[sentence_scores.index(max(sentence_scores))] = -1
    return '。'.join(summary)

In [45]:
summary = get_summary("提请假释建议书2016年娄底监狱假字第46号罪犯王康，男，1987年10月23日出生，汉族，初中文化，湖南省湘乡市人，现在娄底监狱服刑。湖南省长沙市开福区人民法院于二0一一年一月十二日作出（2010）开刑初字第475号刑事判决，以该犯犯盗窃罪，判处有期徒刑十一年，并处罚金三万元，剥夺政治权利一年，刑期从2010年6月10日起至2021年6月9日止。在法定期限内没有上诉、抗诉。2011年1月27日交付执行。服刑期间刑期变动情况：2014年7月1日减去有期徒刑一年八个月，剥夺政治权利一年不变，减刑后刑期至2019年10月9日止。该犯在服刑改造期间，确有悔改表现，并已服够法定年限，具体事实如下：该犯自上次减刑以来，能认罪服法，服从管教，能较好地遵守监规监纪。积极参加\"三课\"学习，态度端正，目的明确。在思想教育中，能联系自己的实际，深挖犯罪根源；在文化和技术学习中，认真好学，成绩较好。在劳动改造中，积极肯干。服从管理民警安排，能按时、保质、保量完成自己的任务。经考核，该犯现累计奖励分158.5分。期内获得单项奖励分30.6分，2014年3月27日因被评为2013年度优秀互监组长奖奖励分3分，2015年2月28日因2014年度实现“七个不发生”奖奖励分3.3分，2016年2月18日因2015年度实现“四无”奖奖励分3分，2016年3月21日因2015年度实现“七个不发生”奖奖励分10.3分。期内扣奖励分1分。上述事实，有罪犯考核奖惩统计台账、罪犯奖惩审批表、罪犯减刑评议书等材料证实。综上所述，该犯在服刑期间确有悔改表现，为此", 100)

In [46]:
summary.__len__(), summary

(2018,
 '湖南省长沙市开福区人民法院于二0一一年一月十二日作出（2010）开刑初字第475号刑事判决。剥夺政治权利一年不变。有罪犯考核奖惩统计台账、罪犯奖惩审批表、罪犯减刑评议书等材料证实。剥夺政治权利一年。服刑期间刑期变动情况：2014年7月1日减去有期徒刑一年八个月。湖南省湘乡市人。判处有期徒刑十一年。该犯在服刑期间确有悔改表现。具体事实如下：该犯自上次减刑以来。提请假释建议书2016年娄底监狱假字第46号罪犯王康。能按时、保质、保量完成自己的任务。积极参加"三课"学习。减刑后刑期至2019年10月9日止。现在娄底监狱服刑。该犯在服刑改造期间。服从管教。在思想教育中。经考核。并处罚金三万元。确有悔改表现。服从管理民警安排。深挖犯罪根源；在文化和技术学习中。该犯现累计奖励分158.5分。在劳动改造中。上述事实。能认罪服法。能较好地遵守监规监纪。积极肯干。期内获得单项奖励分30.6分。2014年3月27日因被评为2013年度优秀互监组长奖奖励分3分。2015年2月28日因2014年度实现“七个不发生”奖奖励分3.3分。2016年2月18日因2015年度实现“四无”奖奖励分3分。2016年3月21日因2015年度实现“七个不发生”奖奖励分10.3分。期内扣奖励分1分。认真好学。2011年1月27日交付执行。刑期从2010年6月10日起至2021年6月9日止。在法定期限内没有上诉、抗诉。初中文化。汉族。男。1987年10月23日出生。以该犯犯盗窃罪。并已服够法定年限。态度端正。目的明确。能联系自己的实际。成绩较好。综上所述。为此。提请假释建议书2016年娄底监狱假字第46号罪犯王康。提请假释建议书2016年娄底监狱假字第46号罪犯王康。提请假释建议书2016年娄底监狱假字第46号罪犯王康。提请假释建议书2016年娄底监狱假字第46号罪犯王康。提请假释建议书2016年娄底监狱假字第46号罪犯王康。提请假释建议书2016年娄底监狱假字第46号罪犯王康。提请假释建议书2016年娄底监狱假字第46号罪犯王康。提请假释建议书2016年娄底监狱假字第46号罪犯王康。提请假释建议书2016年娄底监狱假字第46号罪犯王康。提请假释建议书2016年娄底监狱假字第46号罪犯王康。提请假释建议书2016年娄底监狱假字第46号罪犯王康。提请假释建议书2016年娄底监狱假字第46号罪犯王

In [1]:
from textrank4zh import TextRank4Sentence, TextRank4Keyword
text = "提请假释建议书2016年娄底监狱假字第46号罪犯王康，男，1987年10月23日出生，汉族，初中文化，湖南省湘乡市人，现在娄底监狱服刑。湖南省长沙市开福区人民法院于二0一一年一月十二日作出（2010）开刑初字第475号刑事判决，以该犯犯盗窃罪，判处有期徒刑十一年，并处罚金三万元，剥夺政治权利一年，刑期从2010年6月10日起至2021年6月9日止。在法定期限内没有上诉、抗诉。2011年1月27日交付执行。服刑期间刑期变动情况：2014年7月1日减去有期徒刑一年八个月，剥夺政治权利一年不变，减刑后刑期至2019年10月9日止。该犯在服刑改造期间，确有悔改表现，并已服够法定年限，具体事实如下：该犯自上次减刑以来，能认罪服法，服从管教，能较好地遵守监规监纪。积极参加\"三课\"学习，态度端正，目的明确。在思想教育中，能联系自己的实际，深挖犯罪根源；在文化和技术学习中，认真好学，成绩较好。在劳动改造中，积极肯干。服从管理民警安排，能按时、保质、保量完成自己的任务。经考核，该犯现累计奖励分158.5分。期内获得单项奖励分30.6分，2014年3月27日因被评为2013年度优秀互监组长奖奖励分3分，2015年2月28日因2014年度实现“七个不发生”奖奖励分3.3分，2016年2月18日因2015年度实现“四无”奖奖励分3分，2016年3月21日因2015年度实现“七个不发生”奖奖励分10.3分。期内扣奖励分1分。上述事实，有罪犯考核奖惩统计台账、罪犯奖惩审批表、罪犯减刑评议书等材料证实。综上所述，该犯在服刑期间确有悔改表现，为此"
tr4s = TextRank4Sentence()
tr4s.analyze(text=text, lower=True, source = 'all_filters')

print()
print( '摘要：' )
for item in tr4s.get_key_sentences(num=3):
    print(item.index, item.weight, item.sentence)  # index是语句在文本中位置，weight是权重


Building prefix dict from the default dictionary ...
Loading model from cache /tmp/jieba.cache
Loading model cost 0.686 seconds.
Prefix dict has been built successfully.


AttributeError: module 'networkx' has no attribute 'from_numpy_matrix'

In [3]:
import json

file1 = "../data/train_2.json"
file2 = "../data/valid_2.json"

datalist = []
with open(file1, 'r', encoding='utf8') as f1:
    datalist += list(json.load(f1))
with open(file2, 'r', encoding='utf8') as f2:
    datalist += list(json.load(f2))
datalist.__len__()
    

51458

In [4]:
import pickle

charge2id = {}
id2charge = {}
idx = 0
for item in datalist:
    charges = item['charge']
    for charge in charges:
        if charge not in charge2id:
            charge2id[charge] = idx
            id2charge[idx] = charge
            idx += 1
pickle.dump(charge2id, open("../data/charge2id.pkl", 'wb'))
pickle.dump(id2charge, open("../data/id2charge.pkl", 'wb'))

In [7]:
charge2id.values()

dict_values([0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116, 117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129, 130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142, 143, 144, 145, 146, 147, 148, 149, 150, 151, 152, 153, 154, 155, 156, 157, 158, 159, 160, 161, 162])

In [12]:
from transformers import AutoModel

model = AutoModel.from_pretrained("../../plms/macbert")

model

Some weights of the model checkpoint at ../../plms/macbert were not used when initializing BertModel: ['cls.seq_relationship.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


BertModel(
  (embeddings): BertEmbeddings(
    (word_embeddings): Embedding(21128, 768, padding_idx=0)
    (position_embeddings): Embedding(512, 768)
    (token_type_embeddings): Embedding(2, 768)
    (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (encoder): BertEncoder(
    (layer): ModuleList(
      (0): BertLayer(
        (attention): BertAttention(
          (self): BertSelfAttention(
            (query): Linear(in_features=768, out_features=768, bias=True)
            (key): Linear(in_features=768, out_features=768, bias=True)
            (value): Linear(in_features=768, out_features=768, bias=True)
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (output): BertSelfOutput(
            (dense): Linear(in_features=768, out_features=768, bias=True)
            (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
            (dropout): Dropout(p=0.1, inplace=False)
          